<a href="https://colab.research.google.com/github/gabbypaola/COOP/blob/main/COOP_Race_Data_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt

#connect notebook to google drive to access CSV files
from google.colab import drive
drive.mount('/content/drive/')

#authenticate, needed for loading google sheets into the notebook
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#load worksheet from google drive
race_2019 = gc.open('RACE2019').sheet1

# get_all_values gives a list of rows.
rows9 = race_2019.get_all_values()
#print(rows)

race_data3 = pd.DataFrame.from_records(rows9)
#################################################
col_list = race_data3.iloc[0]

col_list[3::4]=col_list[1::4]

race_data3.iloc[0] = col_list

new_header = race_data3.iloc[1] #grab the first row for the header
race_data3 = race_data3[0:] #take the data less the header row
race_data3.columns = new_header #set the header row as the df header
race_data3 = race_data3.drop(1)
filter = ['Label','Percent']
race_data3 = race_data3[race_data3.columns[race_data3.columns.isin(filter)]]
new_header = race_data3.iloc[0] #grab the first row for the header
race_data3 = race_data3[1:] #take the data less the header row
race_data3.columns = new_header #set the header row as the df header

column_names = []
for i in race_data3.columns[1:]:
  column_names.append(i[6:11])

original_col_names = list(race_data3.columns[1:])

col_names_dict = {original_col_names[i]: column_names[i] for i in range(len(original_col_names))}

race_data3= race_data3.rename(columns=col_names_dict)

race19 = race_data3.loc[[2,3,37,38,40,45]]
race19 = race19.melt(id_vars=[''], 
        var_name="Zip Code", 
        value_name="Percent")
race19 = race19.rename(columns = {'':'Demographic'})

In [ ]:
# 19,310,616 rows of data includes all zip codes for the state of NY
race19['Year'] = 2019

In [ ]:
race19

,Demographic,Zip Code,Percent,Year
0,Male,06390,60.0%,2019
1,Female,06390,40.0%,2019
2,White,06390,93.6%,2019
3,Black or African American,06390,3.2%,2019
4,Asian,06390,0.0%,2019
...,...,...,...,...
19310611,Female,14905,55.3%,2019
19310612,White,14905,89.5%,2019
19310613,Black or African American,14905,8.2%,2019
19310614,Asian,14905,2.0%,2019


In [ ]:
race19['Zip Code']=race19['Zip Code'].astype(str).astype(int)

Load Eviction Zip Codes for NYC

In [ ]:
#load the original data set
zip_codes =pd.read_csv("/content/drive/My Drive/COOP/Data Analytics/Project 1/ZipCodes.csv")

zip_codes['Zip Code'] =zip_codes['Zip Code'].astype(str).astype(int)

In [ ]:
demo_2019 = pd.merge(zip_codes, race19, how= 'inner', on = 'Zip Code')
demo_2019 = demo_2019.drop_duplicates()

In [ ]:
# 1,092 zip codes for NYC data
demo_2019

,Zip Code,Demographic,Percent,Year
0,10001,Male,50.5%,2019
1,10001,Female,49.5%,2019
2,10001,White,67.7%,2019
3,10001,Black or African American,8.3%,2019
4,10001,Asian,23.1%,2019
...,...,...,...,...
5769505,11424,Female,-,2019
5769506,11424,White,-,2019
5769507,11424,Black or African American,-,2019
5769508,11424,Asian,-,2019


In [ ]:
demo1718 =pd.read_csv("/content/drive/My Drive/COOP/Data Analytics/Project 1/demo1718.csv")

In [ ]:
demo1718

,Zip Code,Demographic,Percent,Year
0,10001,Male,50.6%,2017
1,10001,Female,49.4%,2017
2,10001,White,64.3%,2017
3,10001,Black or African American,11.4%,2017
4,10001,Asian,22.8%,2017
...,...,...,...,...
2179,11424,Female,-,2018
2180,11424,White,-,2018
2181,11424,Black or African American,-,2018
2182,11424,Asian,-,2018


In [ ]:
demo171819 = pd.concat([demo1718, demo_2019], axis=0)

In [ ]:
demo171819['Percent']=demo171819['Percent'].str.rstrip('%').replace('-', np.NAN).astype('float')

In [ ]:
demo171819

,Zip Code,Demographic,Percent,Year
0,10001,Male,50.6,2017
1,10001,Female,49.4,2017
2,10001,White,64.3,2017
3,10001,Black or African American,11.4,2017
4,10001,Asian,22.8,2017
...,...,...,...,...
5769505,11424,Female,NaN,2019
5769506,11424,White,NaN,2019
5769507,11424,Black or African American,NaN,2019
5769508,11424,Asian,NaN,2019


In [ ]:
demo171819.isna().sum()

Zip Code        0
Demographic     0
Percent        54
Year            0
dtype: int64

In [ ]:
demo171819.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3276 entries, 0 to 5769509
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Zip Code     3276 non-null   int64  
 1   Demographic  3276 non-null   object 
 2   Percent      3222 non-null   float64
 3   Year         3276 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 128.0+ KB


In [ ]:
zip_boro =pd.read_csv("/content/drive/My Drive/COOP/Data Analytics/Project 1/ZipCodes_Borough.csv")

In [ ]:
demo_final = pd.merge(demo171819, zip_boro, how= 'inner', on = 'Zip Code')

In [ ]:
demo_final=demo_final.drop_duplicates()

In [ ]:
demo_final

,Zip Code,Demographic,Percent,Year,BOROUGH
0,10001,Male,50.6,2017,MANHATTAN
3,10001,Female,49.4,2017,MANHATTAN
6,10001,White,64.3,2017,MANHATTAN
9,10001,Black or African American,11.4,2017,MANHATTAN
12,10001,Asian,22.8,2017,MANHATTAN
...,...,...,...,...,...
9661,11424,Female,NaN,2019,QUEENS
9662,11424,White,NaN,2019,QUEENS
9663,11424,Black or African American,NaN,2019,QUEENS
9664,11424,Asian,NaN,2019,QUEENS


In [ ]:
demo_final['Demographic'] = demo_final['Demographic'].replace('Black or African American','Black')

In [ ]:
demo_final['Demographic'] = demo_final['Demographic'].replace('Hispanic or Latino (of any race)','Hispanic')

In [ ]:
demo_final

,Zip Code,Demographic,Percent,Year,BOROUGH
0,10001,Male,50.6,2017,MANHATTAN
3,10001,Female,49.4,2017,MANHATTAN
6,10001,White,64.3,2017,MANHATTAN
9,10001,Black,11.4,2017,MANHATTAN
12,10001,Asian,22.8,2017,MANHATTAN
...,...,...,...,...,...
9661,11424,Female,NaN,2019,QUEENS
9662,11424,White,NaN,2019,QUEENS
9663,11424,Black,NaN,2019,QUEENS
9664,11424,Asian,NaN,2019,QUEENS


In [ ]:
#export final CSV file to google drive
demo_final.to_csv('/content/drive/My Drive/COOP/Data Analytics/Team Folder/Demographics.csv', index=False)